In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBRegressor

import matplotlib as plt

In [2]:
import matplotlib.pyplot as plt

In [3]:
from xgboost.sklearn import XGBClassifier

In [4]:
directory = "C:\\Users\\shenbaga.kumar\\Google Drive\\Hadoop Training\\Spark Modelling Projects\\"

In [5]:
abcde_rawdata = pd.read_csv(directory + 'abcde Raw Data.txt', delimiter="\t")

In [7]:
abcde_rawdata.shape

(199733, 301)

In [8]:
abcde_rawdata.LoanMonth_Name.unique()

array(['Aug'], dtype=object)

In [9]:
#drop the LoanMonth and the account number columns
abcde_rawdata.drop('LoanMonth_Name',axis=1, inplace=True)

In [10]:
abcde_rawdata.drop('account_number',axis=1, inplace=True)

In [11]:
object_cols = []

In [12]:
#get the list of object columns and encode them:
for x in abcde_rawdata.columns:
    if abcde_rawdata[x].dtype == 'object':
        object_cols.append(x)
        abcde_rawdata[x] = pd.Categorical(abcde_rawdata[x]).codes

In [13]:
pd.set_option('display.max_columns', None)

In [15]:
abcde_rawdata.adopted_Y_N.value_counts()

0    195861
1      3872
Name: adopted_Y_N, dtype: int64

In [16]:
features = list(abcde_rawdata.columns.values)

In [18]:
features.remove('adopted_Y_N')

In [19]:
#split data in to train and test
train = abcde_rawdata.sample(frac=0.8, random_state=0)
test = abcde_rawdata.drop(train.index)

In [20]:
y_train = train.adopted_Y_N

In [ ]:
#http://education.parrotprediction.teachable.com/courses/practical-xgboost-in-python/lectures/1173895
#notes on handling imbalancced data:
#min child weight = 1 is preferred for imbalanced data
#weights while initializing DMatrix - how to sklearn?
#use metrics like AUC

In [21]:
#approach 1: set min child weight to 1 and find the conf matrix

In [23]:
#parameters

sklearn_classfr_params1 = {'learning_rate': 0.1,
                          'subsample': 0.8,
                          'n_estimators': 1000,
                          'seed':0,
                          'colsample_bytree': 0.8,
                          'objective': 'binary:logistic',
                          'max_depth': 3,
                          'min_child_weight': 1
                         }

sklearn_xgb = XGBClassifier(**sklearn_classfr_params1)

In [24]:
sklearn_xgb.fit(train[features], y_train)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=1000, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.8)

In [36]:
test_probabilities = sklearn_xgb.predict_proba(data=test[features])

In [37]:
test_probabilities

array([[ 0.99787593,  0.00212405],
       [ 0.9873023 ,  0.01269771],
       [ 0.99791175,  0.00208827],
       ..., 
       [ 0.99656504,  0.00343495],
       [ 0.96181393,  0.03818606],
       [ 0.99758321,  0.00241678]], dtype=float32)

In [45]:
for i in range(len(test_preds)):
    if test_preds[i] == 1:
        print i

50
88
162
190
293
391
422
531
569
609
750
770
855
859
920
954
968
998
1129
1282
1372
1413
1444
1543
1643
1659
1776
1844
1854
1900
1912
1922
2154
2325
2341
2353
2429
2599
2616
2693
2857
3104
3117
3180
3872
3986
4111
4118
4322
4439
4610
4633
4709
4834
5053
5164
5208
5234
5236
5454
5478
5605
5799
5817
5901
5999
6245
6272
6339
6546
6573
6644
6736
7175
7499
7637
7646
7680
7771
7834
7940
7962
8010
8021
8209
8416
8545
8602
8701
8736
8782
8795
8873
8959
8995
9041
9074
9155
9301
9443
9510
9530
9739
9910
10159
10187
10190
10362
10391
10477
10600
10668
10704
10749
10798
10799
11173
11416
11623
11847
11880
11990
11994
12014
12235
12273
12301
12520
12626
13065
13120
13182
13255
13391
13464
13525
13576
13668
13675
13743
13972
14036
14102
14196
14254
14528
14751
14786
14825
14867
15001
15063
15152
15242
15359
15360
15386
15472
15486
15503
15535
15657
15674
15746
15867
15889
15972
15993
16019
16028
16047
16210
16585
16586
16671
16682
16707
16809
16812
16889
17028
17055
17271
17437
17457
17530
17706
17

In [47]:
for i in range(49,89):
    print i,test_preds[i], test_probabilities[i]

49 0 [ 0.99886441  0.00113557]
50 1 [ 0.00157213  0.99842787]
51 0 [ 0.99844807  0.00155195]
52 0 [ 0.99584401  0.00415598]
53 0 [ 0.9986915   0.00130851]
54 0 [ 0.99564666  0.00435332]
55 0 [  9.99577284e-01   4.22740122e-04]
56 0 [ 0.98866737  0.01133262]
57 0 [ 0.94350475  0.05649527]
58 0 [ 0.9817124   0.01828762]
59 0 [  9.99899089e-01   1.00891928e-04]
60 0 [ 0.74476761  0.25523239]
61 0 [ 0.92397195  0.07602807]
62 0 [ 0.99740028  0.00259973]
63 0 [ 0.98320091  0.01679911]
64 0 [  9.99495268e-01   5.04744763e-04]
65 0 [  9.99776959e-01   2.23042196e-04]
66 0 [ 0.99886262  0.00113739]
67 0 [ 0.9581874  0.0418126]
68 0 [ 0.99624389  0.00375611]
69 0 [ 0.99716765  0.00283238]
70 0 [ 0.97995561  0.02004439]
71 0 [ 0.99705768  0.00294234]
72 0 [ 0.99567789  0.00432213]
73 0 [  9.99906123e-01   9.38890371e-05]
74 0 [ 0.99087101  0.00912899]
75 0 [  9.99929130e-01   7.08546868e-05]
76 0 [  9.99700487e-01   2.99508392e-04]
77 0 [ 0.99673051  0.00326947]
78 0 [ 0.99476957  0.0052304 ]
79

In [56]:
test_preds=np.zeros(len(test_probabilities))

In [48]:
len(test_probabilities)

39947

In [59]:
for x in range(len(test_probabilities)):
    if test_probabilities[x][1] >= 0.40:
        test_preds[x] = 1

In [60]:
sum(test_preds)

39947.0

In [61]:
test_preds_frm_model = sklearn_xgb.predict(data=test[features])

In [62]:
sum(test_preds_frm_model)

390

In [63]:
ntest = test.shape[0]

In [64]:
ntest

39947

In [65]:
#so if we use the predict function, we see that it predicts 390 1s
#lets use the predicted probabilities and a lower threshold of 0.4 to see if the 1s coverage increases

In [66]:
test_probabilities = sklearn_xgb.predict_proba(test[features])

In [67]:
test_probabilities

array([[ 0.99787593,  0.00212405],
       [ 0.9873023 ,  0.01269771],
       [ 0.99791175,  0.00208827],
       ..., 
       [ 0.99656504,  0.00343495],
       [ 0.96181393,  0.03818606],
       [ 0.99758321,  0.00241678]], dtype=float32)

In [71]:
test_preds_0point4_thres = np.zeros(len(test_probabilities))

In [73]:
test_preds_0point4_thres, len(test_preds_0point4_thres)

(array([ 0.,  0.,  0., ...,  0.,  0.,  0.]), 39947)

In [74]:
#create a new array with the predictions based on lowered threshold
for i in range(ntest):
    if test_probabilities[i][1] >= 0.4:
        test_preds_0point4_thres[i] = 1
    else:
        test_preds_0point4_thres[i] = 0

In [75]:
sum(test_preds_0point4_thres)

440.0

In [76]:
#the coverage marginally increases to 440 from 397

In [77]:
pd.crosstab(
    pd.Series(test.adopted_Y_N, name='Actual'),
    pd.Series(test_preds_0point4_thres, name='Predicted'),
    margins=True
)

Predicted,0.0,1.0,All
Actual,,,
0.0,7854,86,7940
1.0,164,1,165
All,8018,87,8105


In [78]:
test.adopted_Y_N.value_counts()

0    39180
1      767
Name: adopted_Y_N, dtype: int64

In [81]:
test.adopted_Y_N = test.adopted_Y_N.values.astype(float)

In [82]:
test.adopted_Y_N.value_counts()

0.0    39180
1.0      767
Name: adopted_Y_N, dtype: int64

In [83]:
pd.crosstab(
    pd.Series(test.adopted_Y_N, name='Actual'),
    pd.Series(test_preds_0point4_thres, name='Predicted'),
    margins=True
)

Predicted,0.0,1.0,All
Actual,,,
0.0,7854,86,7940
1.0,164,1,165
All,8018,87,8105


In [84]:
#not sure why we are getting only 8105 out of the total of 39947!!

In [85]:
len( pd.Series(test_preds_0point4_thres, name='Predicted'))

39947

In [86]:
len(pd.Series(test.adopted_Y_N, name='Actual'))

39947

In [91]:
from sklearn.metrics import confusion_matrix

In [93]:
test.adopted_Y_N.value_counts()

0.0    39180
1.0      767
Name: adopted_Y_N, dtype: int64

In [92]:
confusion_matrix(y_true=test.adopted_Y_N , y_pred=test_preds_0point4_thres)

array([[39126,    54],
       [  381,   386]])

In [94]:
#so with a threshold of 0.4, we have classified 386 of the 767 as 1s
#also print the other metrics

In [95]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [99]:
print "accuracy is: " + str(accuracy_score(y_true=test.adopted_Y_N , y_pred=test_preds_0point4_thres))
print "precision is: " + str(precision_score(y_true=test.adopted_Y_N , y_pred=test_preds_0point4_thres))
print "recall is: " + str(recall_score(y_true=test.adopted_Y_N , y_pred=test_preds_0point4_thres))

accuracy is: 0.989110571507
precision is: 0.877272727273
recall is: 0.503259452412


In [100]:
#while accuracy and precision are high, recall is low (50%) which means that we are poor in predicting 1s

In [101]:
#approach 2: adjust  the scale pos weight 

In [106]:
#get ratio of 0s to 1s
ratio = float(np.sum(train.adopted_Y_N == 0)) / np.sum(train.adopted_Y_N == 1)

In [107]:
sklearn_classfr_params2 = {'learning_rate': 0.1,
                          'subsample': 0.8,
                          'n_estimators': 1000,
                          'seed':0,
                          'colsample_bytree': 0.8,
                          'objective': 'binary:logistic',
                          'max_depth': 3,
                          'min_child_weight': 1,
                            'scale_pos_weight': ratio #this is the only addition to the first params dict
                         }


In [109]:
sklearn_xgb2 = XGBClassifier(**sklearn_classfr_params2)

In [110]:
sklearn_xgb2

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=1000, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=50.460869565217394, seed=0, silent=True,
       subsample=0.8)

In [111]:
sklearn_xgb2.fit(train[features], y_train)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=1000, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=50.460869565217394, seed=0, silent=True,
       subsample=0.8)

In [112]:
test_preds2 = sklearn_xgb2.predict(test[features])

In [113]:
sum(test_preds2)

1975

In [114]:
#we see a clear improvement in the number of 1s without changing the thres itself
#now change the thres and see the new preds

In [115]:
test_probs2 = sklearn_xgb2.predict_proba(test[features])

In [116]:
test_preds2_0point4_thres = np.zeros(len(test_probs2))

In [117]:
#create a new array with the predictions based on lowered threshold
for i in range(ntest):
    if test_probs2[i][1] >= 0.4:
        test_preds2_0point4_thres[i] = 1
    else:
        test_preds2_0point4_thres[i] = 0

In [118]:
sum(test_preds2_0point4_thres)

2646.0

In [119]:
#a significant improvement in 1s to 2646 from 1975

In [120]:
confusion_matrix(y_true=test.adopted_Y_N , y_pred=test_preds2_0point4_thres)

array([[37158,  2022],
       [  143,   624]])

In [121]:
#we see a significant improvement in the prediction of 1s but 
#that comes at the cost of more 0s predicted (inaccurately) as 1

In [122]:
print "accuracy is: " + str(accuracy_score(y_true=test.adopted_Y_N , y_pred=test_preds2_0point4_thres))
print "precision is: " + str(precision_score(y_true=test.adopted_Y_N , y_pred=test_preds2_0point4_thres))
print "recall is: " + str(recall_score(y_true=test.adopted_Y_N , y_pred=test_preds2_0point4_thres))

accuracy is: 0.945803189226
precision is: 0.235827664399
recall is: 0.813559322034


In [123]:
#same point proven by the above metrics, recall has increased at the cost of preicision

In [124]:
#approach 3: adjust the weight parameter, sample weight (??) in the sklearn version

In [125]:
sklearn_xgb3 = XGBClassifier(**sklearn_classfr_params2)

In [126]:
#weights based on the 1s and 0s
weights = np.zeros(len(y_train))
weights[y_train == 0] = 1
weights[y_train == 1] = 5

C:\Users\shenbaga.kumar\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index
  app.launch_new_instance()
C:\Users\shenbaga.kumar\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index


In [127]:
weights

array([ 5.,  5.,  0., ...,  0.,  0.,  0.])

In [128]:
sklearn_xgb3.fit(X=train[features], y=y_train,sample_weight=weights)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=1000, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=50.460869565217394, seed=0, silent=True,
       subsample=0.8)

In [129]:
test_preds3 = sklearn_xgb3.predict(test[features])

In [130]:
sum(test_preds3)

0

In [132]:
#something is off

In [133]:
sum(sklearn_xgb3.predict(train[features]))

0

In [135]:
#try the weights using the native API
#first prepare the DMatrix
dtrain = xgb.DMatrix(train[features], label=y_train, weight=weights)
dtest = xgb.DMatrix(test[features])

In [136]:
#define params for the model:
native_params = {'eta': 0.1,
                  'subsample': 0.8,
                  'n_estimators': 1000,
                  'seed':0,
                  'colsample_bytree': 0.8,
                  'objective': 'binary:logistic',
                  'max_depth': 3,
                  'min_child_weight': 1}

In [137]:
xgb_native_model = xgb.train(params=native_params, dtrain=dtrain,num_boost_round=15)

In [138]:
test_preds_xgb_native = xgb_native_model.predict(dtest)

In [139]:
test_preds_xgb_native

array([ 0.21516816,  0.21516816,  0.21516816, ...,  0.21516816,
        0.21516816,  0.21516816], dtype=float32)

In [140]:
test_preds_0_1xgb_native = (test_preds_xgb_native > 0.4).astype(int)

In [141]:
test_preds_0_1xgb_native

array([0, 0, 0, ..., 0, 0, 0])

In [142]:
sum(test_preds_0_1xgb_native)

0

In [143]:
#same result as the sklearn model, so weights is not working 

In [144]:
sklearn_xgb3.get_params

<bound method XGBClassifier.get_params of XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=1000, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=50.460869565217394, seed=0, silent=True,
       subsample=0.8)>

In [154]:
#how to get AUC as the eval metric for xgb
sklearn_classfr_params4 = {'learning_rate': 0.1,
                          'subsample': 0.8,
                          'n_estimators': 1000,
                          'seed':0,
                          'colsample_bytree': 0.8,
                          'objective': 'binary:logistic',
                          'max_depth': 3,
                          'min_child_weight': 1,
                          'scale_pos_weight': ratio, #this is the only addition to the first params dict
                          # 'objective': 'auc' #added for this param; not sure if this corresponds to the eval metric
                           #objective is not the param; it throws errors
                         }

In [156]:
sklearn_xgb4 = XGBClassifier(**sklearn_classfr_params4)

In [157]:
sklearn_xgb4.fit(train[features], y_train, eval_metric='auc')

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=1000, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=50.460869565217394, seed=0, silent=True,
       subsample=0.8)

In [158]:
test_probs4 = sklearn_xgb4.predict_proba(test[features])

In [159]:
test_preds4 = np.zeros(test.shape[0])

In [160]:
#create a new array with the predictions based on lowered threshold
for i in range(ntest):
    if test_probs4[i][1] >= 0.4:
        test_preds4[i] = 1
    else:
        test_preds4[i] = 0

In [161]:
sum(test_preds4)

2646.0

In [162]:
#same as the earlier model, meaning that auc was actually the eval metric earlier as well (without us explicitly stating it)

#21 Dec 2016: we stop now:


Learnings:

1. Use XGB for the classifier data
2. data is imbalanced, so tried a few techniques to adjust for that in xgb
3. setting scale pos weight to the ratio of 0s to 1s provided the best results for recall, i.e. predicting 1s
4. adjusting the weights param was not successful, could not figure out why